In [2]:
import datetime
import itertools
import json
import os
import time
from collections import defaultdict
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
# import xgboost as xgb
from pandas import Series
from scipy.stats.mstats import winsorize
from sklearn.ensemble import (AdaBoostClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier, RandomForestClassifier)
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
# from statsmodels.tsa.arima_model import ARIMA

from fbprophet import Prophet

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
# sns.set()

In [3]:
merged_csv_path = '../Merge_Data_Non_Negative.csv'
merged_df= pd.read_csv(
        merged_csv_path, low_memory=False, thousands=',')

In [4]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104510 entries, 0 to 104509
Data columns (total 6 columns):
sku_id    104510 non-null object
week1     104510 non-null float64
week2     104510 non-null float64
week3     104510 non-null float64
week4     104510 non-null float64
week5     104510 non-null float64
dtypes: float64(5), object(1)
memory usage: 4.8+ MB


In [5]:
merged_df.describe(include='all')

,sku_id,week1,week2,week3,week4,week5
count,104510,104510.000000,104510.000000,104510.000000,104510.000000,104510.000000
unique,104510,NaN,NaN,NaN,NaN,NaN
top,SKwtcjH3,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.793852,2.709377,3.147550,3.762592,4.036621
std,NaN,11.788133,10.935311,12.351985,15.236028,17.346439
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.277520,0.301464,0.373967,0.413612,0.385814
50%,NaN,0.567460,0.567129,0.667230,0.757246,0.709179
75%,NaN,1.671865,1.649685,1.975092,2.289659,2.282702


In [6]:
best_mean_path = '../mean_submission_52.csv'
best_mean_df = pd.read_csv(best_mean_path, low_memory=False, thousands=',')

In [7]:
best_mean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104510 entries, 0 to 104509
Data columns (total 6 columns):
sku_id    104510 non-null object
week1     104510 non-null float64
week2     104510 non-null float64
week3     104510 non-null float64
week4     104510 non-null float64
week5     104510 non-null float64
dtypes: float64(5), object(1)
memory usage: 4.8+ MB


In [8]:
best_mean_df.describe(include='all')

,sku_id,week1,week2,week3,week4,week5
count,104510,104510.000000,104510.000000,104510.000000,104510.000000,104510.000000
unique,104510,NaN,NaN,NaN,NaN,NaN
top,SKwtcjH3,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.063525,2.063525,2.063525,2.063525,2.063525
std,NaN,8.364585,8.364585,8.364585,8.364585,8.364585
min,NaN,0.134615,0.134615,0.134615,0.134615,0.134615
25%,NaN,0.134615,0.134615,0.134615,0.134615,0.134615
50%,NaN,0.403846,0.403846,0.403846,0.403846,0.403846
75%,NaN,1.346154,1.346154,1.346154,1.346154,1.346154


In [9]:
# plt.plot(best_mean_df.sku_id, best_mean_df.week1)
# plt.show()
# plt.plot(merged_df.sku_id, merged_df.week1)

In [10]:
best_mean_df = best_mean_df.sort_values('sku_id')
merged_df = merged_df.sort_values('sku_id')

In [11]:
best_mean_df.head()

,sku_id,week1,week2,week3,week4,week5
60436,SK013PAq,0.403846,0.403846,0.403846,0.403846,0.403846
33306,SK013i5Y,0.538462,0.538462,0.538462,0.538462,0.538462
45433,SK014m67,1.480769,1.480769,1.480769,1.480769,1.480769
6603,SK016iX3,0.134615,0.134615,0.134615,0.134615,0.134615
45068,SK019vAh,0.807692,0.807692,0.807692,0.807692,0.807692


In [12]:
merged_df.head()

,sku_id,week1,week2,week3,week4,week5
58036,SK013PAq,0.984831,0.939258,0.942489,0.981239,0.998873
27306,SK013i5Y,1.344822,1.648666,1.978070,2.127624,1.990003
41433,SK014m67,0.498950,0.550849,0.636652,0.643121,0.563581
48603,SK016iX3,0.530302,0.498344,0.496245,0.521204,0.536979
41068,SK019vAh,0.589062,0.701647,0.709782,0.607125,0.535784


In [13]:
# blending..
b1 = best_mean_df.copy()
# col = best_mean_df.columns

# col = col.tolist()
# col.remove('sku_id')
# for i in col:
#     b1[i] = 0.95 * best_mean_df[i]

# #SXX
w1 = np.random.uniform(0.75, 0.95) 
w2 = np.random.uniform(0.95, 1.05)
w3 = np.random.uniform(1.8, 2.2)
w4 = np.random.uniform(2.3, 2.8)
w5 = np.random.uniform(0.98, 1.2)
total = 1
cons = 0

result = b1
# result['week1'] = result['week1'] + cons
# result['week2'] = result['week2'] + cons
# result['week3'] = result['week3'] + cons
# result['week4'] = result['week4'] + cons
# result['week5'] = result['week5'] + cons
result['week1'] = result['week1'] * w1 / total
result['week2'] = result['week2'] * w2 / total
result['week3'] = result['week3'] * w3 / total
result['week4'] = result['week4'] * w4 / total
result['week5'] = result['week5'] * w5 / total

b1.to_csv('final-uniform.csv', index=False)

In [14]:
b1.describe(include='all')

,sku_id,week1,week2,week3,week4,week5
count,104510,104510.000000,104510.000000,104510.000000,104510.000000,104510.000000
unique,104510,NaN,NaN,NaN,NaN,NaN
top,SKRv08nT,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.776787,2.105642,4.180757,5.558386,2.243895
std,NaN,7.202281,8.535309,16.946879,22.531156,9.095723
min,NaN,0.115910,0.137363,0.272734,0.362605,0.146382
25%,NaN,0.115910,0.137363,0.272734,0.362605,0.146382
50%,NaN,0.347730,0.412089,0.818203,1.087815,0.439146
75%,NaN,1.159098,1.373629,2.727345,3.626050,1.463819


In [15]:
best_mean_df.describe(include='all')

,sku_id,week1,week2,week3,week4,week5
count,104510,104510.000000,104510.000000,104510.000000,104510.000000,104510.000000
unique,104510,NaN,NaN,NaN,NaN,NaN
top,SKRv08nT,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.063525,2.063525,2.063525,2.063525,2.063525
std,NaN,8.364585,8.364585,8.364585,8.364585,8.364585
min,NaN,0.134615,0.134615,0.134615,0.134615,0.134615
25%,NaN,0.134615,0.134615,0.134615,0.134615,0.134615
50%,NaN,0.403846,0.403846,0.403846,0.403846,0.403846
75%,NaN,1.346154,1.346154,1.346154,1.346154,1.346154


In [16]:
best_mean_df.head(2)

,sku_id,week1,week2,week3,week4,week5
60436,SK013PAq,0.403846,0.403846,0.403846,0.403846,0.403846
33306,SK013i5Y,0.538462,0.538462,0.538462,0.538462,0.538462


In [17]:
b1.head(2)

,sku_id,week1,week2,week3,week4,week5
60436,SK013PAq,0.347730,0.412089,0.818203,1.087815,0.439146
33306,SK013i5Y,0.463639,0.549452,1.090938,1.450420,0.585528


In [23]:
b1['week1'][60436] / best_mean_df['week1'][60436]

0.8610445991028529

In [24]:
b1['week2'][60436] / best_mean_df['week2'][60436]

1.0204102853493582

In [25]:
b1['week3'][60436] / best_mean_df['week3'][60436]

2.0260273577387697

In [26]:
b1['week4'][60436] / best_mean_df['week4'][60436]

2.6936369468378225

In [27]:
b1['week5'][60436] / best_mean_df['week5'][60436]

1.0874086995420913

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

cv_params = {
    # 'n_estimators': range(160, 200, 3),
    # 'max_depth': [4, 5, 6, 7],
    # 'min_child_weight': [1, 2],
    # 'gamma': [i / 30.0 for i in range(0, 5)],
    # 'subsample': [i * .01 for i in range(80, 90, 1)],
    # 'colsample_bytree': [i * .01 for i in range(80, 90, 1)],
    # 'reg_alpha': [0.0001, 0.001, 0.005, 0.01],
    # 'reg_lambda': [1e-6, 1e-5, 1e-4, 1e-3],
    # 'learning_rate': [0.1 + 0.1 * i for i in range(10)],
}
other_params = {
    'n_estimators': 5000,
    'max_depth': 5,
    'min_child_weight': 1,
    'gamma': 0.0,
    'subsample': 0.86,
    'colsample_bytree': 0.84,
    'reg_alpha': .0001,
    'reg_lambda': 1e-6,

    'seed': 42,

    'learning_rate': 0.05,
}

fit_params = {
    "early_stopping_rounds": 50,
    "eval_set": [(X, Y)],
}
model = XGBClassifier(**other_params)
model.fit(X, Y)
print("xgb accuracy: {}".format(model.score(X, Y)))
y_pred = model.predict(test)
grid_search = GridSearchCV(estimator=model, param_grid=cv_params, fit_params=fit_params, cv=5,
                           verbose=1, n_jobs=4)
grid_search.fit(X, Y)
print('每轮迭代运行结果: {0}'.format(grid_search.cv_results_))
print('参数的最佳取值：{0}'.format(grid_search.best_params_))
print('最佳模型得分: {0}'.format(grid_search.best_score_))
y_pred = grid_search.predict(test)

In [ ]:
# blending..
b1 = best_mean_df.copy()
# col = best_mean_df.columns

# col = col.tolist()
# col.remove('sku_id')
# for i in col:
#     b1[i] = 0.95 * best_mean_df[i]

# #SXX
w1 = np.random.uniform(0.75, 0.95) 
w2 = np.random.uniform(0.95, 1.05)
w3 = np.random.uniform(1.8, 2.2)
w4 = np.random.uniform(2.3, 2.8)
w5 = np.random.uniform(0.98, 1.2)
total = 1
cons = 0

result = b1
# result['week1'] = result['week1'] + cons
# result['week2'] = result['week2'] + cons
# result['week3'] = result['week3'] + cons
# result['week4'] = result['week4'] + cons
# result['week5'] = result['week5'] + cons
result['week1'] = result['week1'] * w1 / total
result['week2'] = result['week2'] * w2 / total
result['week3'] = result['week3'] * w3 / total
result['week4'] = result['week4'] * w4 / total
result['week5'] = result['week5'] * w5 / total

b1.to_csv('final-uniform.csv', index=False)